<a href="https://colab.research.google.com/github/iisc-ts-group6/capstone/blob/siri/Package_Capstone_Group6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated answer evaluation

**1. Installing and importing packages**

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain-openai
!pip install pypdf
!pip install chromadb
!pip install tiktoken
!pip install langchain-community

ERROR: Could not find a version that satisfies the requirement langchain-communityv (from versions: none)
ERROR: No matching distribution found for langchain-communityv


In [ ]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install langchainhub

Imports

In [ ]:
import os
import openai
import numpy as np

**Authentication for OpenAI API**

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

**Loading the documents**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# Load PDF pypdf2
loaders = [
    # Duplicate documents on purpose
    PyPDFLoader("/content/ncertbio.pdf"),

]
docs = []
for loader in loaders:
    docs.extend(loader.load())

**print them**

In [ ]:
print(docs[0].page_content)

As we look at our surr oundings, we see a lar ge
variety of things with dif ferent shapes, sizes
and textures. Everything in this universe is
made up of material which scientists have
named “matter”. The air we br eathe, the food
we eat, stones, clouds, stars, plants and
animals, even a small dr op of water or a
particle of sand — every thing is matter . We
can also see as we look around that all the
things mentioned above occupy space and
have mass. In other words, they have both
mass* and volume**.
Since early times, human beings have
been trying to understand their surr oundings.
Early Indian philosophers  classified matter in
the form of five basic elements — the “Panch
Tatva” — air , earth, fir e, sky and water .
Accor
ding to them everything, living or non-
living, was made up of these five basic
elements. Ancient Gr eek philosophers had
arrived at a similar classification of matter .
Moder n day scientists have evolved two
types of classification of matter based on their
physica

# Splitting of document

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Split
#from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [ ]:
splits = text_splitter.split_documents(docs)
print(len(splits))
splits

73


[Document(page_content='As we look at our surr oundings, we see a lar ge\nvariety of things with dif ferent shapes, sizes\nand textures. Everything in this universe is\nmade up of material which scientists have\nnamed “matter”. The air we br eathe, the food\nwe eat, stones, clouds, stars, plants and\nanimals, even a small dr op of water or a\nparticle of sand — every thing is matter . We\ncan also see as we look around that all the\nthings mentioned above occupy space and\nhave mass. In other words, they have both', metadata={'source': '/content/ncertbio.pdf', 'page': 0}),
 Document(page_content='have mass. In other words, they have both\nmass* and volume**.\nSince early times, human beings have\nbeen trying to understand their surr oundings.\nEarly Indian philosophers  classified matter in\nthe form of five basic elements — the “Panch\nTatva” — air , earth, fir e, sky and water .\nAccor\nding to them everything, living or non-\nliving, was made up of these five basic\nelements. Ancien

# Embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f948e4925c0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f948e4e3190>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

**Understanding similarity search with a toy example**

In [ ]:
sentence1 = "i like dogs"
sentence2 = "i like cats"
sentence3 = "the weather is ugly, too hot outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
len(embedding1), len(embedding2), len(embedding3)

(1536, 1536, 1536)

In [ ]:
np.dot(embedding1, embedding2), np.dot(embedding1, embedding3),np.dot(embedding2, embedding3)

(0.920009825736222, 0.7636469559970651, 0.7583392710131898)

#Vectorstores

In [ ]:
from langchain_community.vectorstores import Chroma # Light-weight and in memory

In [ ]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits, # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory # save the directory
)

In [ ]:
vectordb.persist() # Let's **save vectordb** so we can use it later!

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
print(vectordb._collection.count()) # same as number of splits

73


# Retrieval + Question Answering : Connecting with LLMs

In [ ]:
llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [ ]:
question = "what are Characteristics of Particles of Matter?"
docs = vectordb.max_marginal_relevance_search(question, k=2, fetch_k=3)
len(docs)

2

In [ ]:
docs[0]

Document(page_content='4.What are the characteristics of\nthe particles of matter?Particles of matter are continuously\nmoving, that is, they possess what we call\nthe kinetic ener gy. As the temperatur e rises,\nparticles move faster . So, we can say that with\nincrease in temperatur e the kinetic ener gy of\nthe particles also incr eases.\nIn the above thr ee activities we observe\nthat particles of matter inter mix on their own\nwith each other . They do so by getting into\nthe spaces between the particles. This', metadata={'page': 2, 'source': '/content/ncertbio.pdf'})

In [ ]:
docs[1]

Document(page_content='As we look at our surr oundings, we see a lar ge\nvariety of things with dif ferent shapes, sizes\nand textures. Everything in this universe is\nmade up of material which scientists have\nnamed “matter”. The air we br eathe, the food\nwe eat, stones, clouds, stars, plants and\nanimals, even a small dr op of water or a\nparticle of sand — every thing is matter . We\ncan also see as we look around that all the\nthings mentioned above occupy space and\nhave mass. In other words, they have both', metadata={'page': 0, 'source': '/content/ncertbio.pdf'})

**Vector store-backed retriever**

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
question = "A diver is able to cut through water in a swimming pool. Which property of matter does this observation show?"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever(), return_source_documents=True)

result = qa_chain.invoke({"query": question})

In [ ]:
result["result"]

'The observation of a diver cutting through water in a swimming pool shows the property of fluidity in matter.'

In [ ]:
result["source_documents"]

[Document(page_content='uestions\n1.Which of the following are\nmatter?\nChair , air, love, smell, hate,\nalmonds, thought, cold, lemon\nwater , smell of per fume.\n2.Give reasons for the following\nobservation:\nThe smell of hot sizzling food\nreaches you several metres\naway, but to get the smell from\ncold food you have to go close.\n3.A diver is able to cut through\nwater in a swimming pool. Which\nproperty of matter does this\nobservation show?\n4.What are the characteristics of', metadata={'page': 2, 'source': '/content/ncertbio.pdf'}),
 Document(page_content='4.Liquids generally have lower\ndensity as compar ed to solids.\nBut you must have observed that\nice floats on water . Find out why.\n1.4Can Matter Change its State?\nWe all know fr om our observation that water\ncan exist in thr ee states of matter–\n•solid, as ice,\n•liquid, as the familiar water , and\n•gas, as water vapour .\nWhat happens inside the matter during\nthis change of state? What happens to the\nparticles of

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [ ]:
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\nQuestion: {question}\nHelpful Answer:")

In [ ]:
# Run chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 2, "fetch_k":6} ), # "k":2, "fetch_k":3
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True
                                       )

In [ ]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\nQuestion: {question}\nHelpful Answer:"), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7dbacf9e8b20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7dbacf9eb6a0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7dbabdb44130>, search_type='mmr', search_kwargs={'k': 2, 'fetch_k': 6}))

In [ ]:
question = "For any substance, why does the temperature remain constant during the change of state?"
result = qa_chain.invoke({"query": question})
result["source_documents"]

[Document(page_content='3.For any substance, why does the\ntemperature remain constant\nduring the change of state?\n4.Suggest a method to liquefy\natmospheric gases.\n1.5 Evaporation\nDo we always need to heat or change pressure\nfor changing the state of matter? Can you\nquote some examples from everyday life where\nchange of state from liquid to vapour takes\nplace without the liquid reaching the boiling\npoint? W ater, when left uncovered, slowly\nchanges into vapour . Wet clothes dry up. What', metadata={'page': 8, 'source': '/content/ncertbio.pdf'}),
 Document(page_content='When a solid melts, its temperature\nremains the same, so where does the heat\nenergy go?\nYou must have observed, during the\nexperiment of melting, that the temperatur e\nof the system does not change after the\nmelting point is reached, till all the ice melts.\nThis happens even though we continue to\nheat the beaker, that is, we continue to supply\nheat. This heat gets used up in changing thestate by overc

In [ ]:
result["result"]

'During a change of state, the temperature remains constant because the heat energy being supplied is used to overcome the forces of attraction between particles, rather than increasing the temperature. This allows the substance to change its state without a change in temperature. Thanks for asking!'

In [ ]:
question = "Convert the following temperature to celsius scale: a. 300 K b. 573 K"
result = qa_chain.invoke({"query": question})
result["result"]
result["source_documents"]

[Document(page_content='SCIENCE 121.Convert the following temperatures to the celsius scale.\n(a)293 K (b)470 K\n2.Convert the following temperatures to the kelvin scale.\n(a)25o°C (b)373o°C\n3.Give reason for the following observations.\n(a)Naphthalene balls disappear with time without leaving\nany solid.\n(b)We can get the smell of perfume sitting several metres\naway.\n4.Arrange the following substances in increasing order of  forces\nof attraction between the particles— water , sugar , oxygen.', metadata={'page': 11, 'source': '/content/ncertbio.pdf'}),
 Document(page_content='373 K (1000 C) have mor e ener gy than water\nat the same temperatur e. This is because\nparticles in steam have absorbed extra ener gy\nin the for m of latent heat of vaporisation.\n*Note: Kelvin is the SI unit of temperature, 0o C =273.15 K. For convenience, we take 0o C = 273 K after\nrounding off the decimal. To change a temperature on the Kelvin scale to the Celsius scale you\nhave to subtract 273 from t

In [ ]:
result["result"]

'a. 300 K = 27°C\nb. 573 K = 300°C'

In [ ]:
student_answer = "(a)300 K = 22°C\n (b) 573 K = 250°C"
llm_answer = result["result"]

In [ ]:
!pip install --upgrade openai

# Compare two sentences using GPT


In [ ]:
from openai import OpenAI

client = OpenAI()
# Function to get the comparison and similarity score from GPT
def compare_sentences(question, student_answer, llm_answer):
    prompt = f"""
    You are an assistant that evaluate a student's answer to a reference answer.

    question: "{question}"
    Student's answer: "{student_answer}"
    Reference answer: "{llm_answer}"

    for the question the correct answer is the "{llm_answer}"
    based on this fact evaluate the "{student_answer}" to be Correct, Partially correct or Incorrect
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4" if you have access
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0
    )


    # Print the raw response for debugging
    #print("Raw response:", response)

    #score = float(response.choices[0].message['content'].strip())
    result = response.choices[0].message
    print(result)
    # Print the extracted result for debugging
    print("Extracted result:", result)

    return response

In [ ]:

# Example sentences
question = "what is needed for photsynthesis?"
student_answer = "photsynthesis process needs corbondioxide and sun light"
llm_answer = "photsynthesis process needs corbondioxide and sun light"
